# Lista prática I

**Instruções gerais:** Sua submissão deve conter: 
1. Um "ipynb" com seu código e as soluções dos problemas
2. Uma versão pdf do ipynp

## Vizinhos mais próximos

**Exercício 1.** O código abaixo carrega o dataset MNIST, que consiste em imagens de dígitos entre $0$ e $9$. Teste o $k$-NN com distância euclidiana para classificação do conjunto de teste. Use valores de $k$ diferentes (e.g., de 1 a 5) e reporte a acurácia para cada valor de $k$. Lembre que a acurácia é o percentual de amostras classificadas corretamente. Notavelmente, as entradas do MNIST tem dimensão relativamente alta (64). Plote uma imagem com a variância amostral dos pixels das imagens e comente. Também mostre as imagens classificadas de maneira errônea e comente.

In [ ]:
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.datasets import load_digits, make_moons
from sklearn.model_selection import train_test_split


SEED = 42
np.random.seed(SEED)

@dataclass
class Dataset:
    features_train: np.ndarray 
    features_test: np.ndarray  
    labels_train: np.ndarray   
    labels_test: np.ndarray

# Import dataset and separate train/test subsets
mnist = Dataset(*train_test_split(
    *load_digits(return_X_y=True),
    random_state=SEED,
))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Definindo a semente para reprodutibilidade
SEED = 42
np.random.seed(SEED)

@dataclass
class Dataset:
    features_train: np.ndarray 
    features_test: np.ndarray  
    labels_train: np.ndarray   
    labels_test: np.ndarray

# Importando o dataset MNIST
mnist = Dataset(*train_test_split(
    *load_digits(return_X_y=True),
    random_state=SEED,
))

# Função para calcular a acurácia e plotar os resultados
def knn_accuracy_for_k_values(k_values, dataset):
    accuracies = []
    for k in k_values:
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(dataset.features_train, dataset.labels_train)
        predictions = model.predict(dataset.features_test)
        accuracies.append(accuracy_score(dataset.labels_test, predictions))
    return accuracies

# Testando o $k$-NN com valores de k de 1 a 5
k_values = range(1, 6)
accuracies = knn_accuracy_for_k_values(k_values, mnist)

# Plotando a acurácia para diferentes valores de k
plt.figure(figsize=(8, 6))
plt.plot(k_values, accuracies, marker='o')
plt.title('Acurácia do $k$-NN para diferentes valores de $k$', fontsize=14)
plt.xlabel('$k$', fontsize=12)
plt.ylabel('Acurácia', fontsize=12)
plt.xticks(k_values)
plt.grid(True)
plt.show()

# Analisando a variância amostral dos pixels
pixel_variance = np.var(mnist.features_train, axis=0)

# Plotando a variância amostral dos pixels
plt.figure(figsize=(8, 6))
sns.heatmap(pixel_variance.reshape(8, 8), cmap='viridis', annot=False, xticklabels=False, yticklabels=False)
plt.title('Variância Amostral dos Pixels das Imagens', fontsize=14)
plt.show()

# Identificando imagens errôneas
model = KNeighborsClassifier(n_neighbors=3)
model.fit(mnist.features_train, mnist.labels_train)
predictions = model.predict(mnist.features_test)

# Encontrando as imagens erradas
incorrect_indices = np.where(predictions != mnist.labels_test)[0]

# Exibindo as imagens erradas
plt.figure(figsize=(12, 12))
for i, idx in enumerate(incorrect_indices[:9]):  # Mostrando até 9 imagens erradas
    plt.subplot(3, 3, i+1)
    plt.imshow(mnist.features_test[idx].reshape(8, 8), cmap='gray')
    plt.title(f'Errado: {predictions[idx]} | Real: {mnist.labels_test[idx]}', fontsize=10)
    plt.axis('off')
plt.suptitle('Imagens Erradas Classificadas pelo Modelo $k$-NN', fontsize=16)
plt.show()


### Explicação do Código:

#### 1. **$k$-NN com diferentes valores de $k$ (1 a 5):**

A função `knn_accuracy_for_k_values` realiza o treinamento do modelo para diferentes valores de $k$, variando entre 1 e 5. Para cada valor de $k$, o modelo é treinado e em seguida realiza-se a previsão no conjunto de teste.

- O **classificador $k$-NN** é implementado através da classe `KNeighborsClassifier` do scikit-learn, que calcula as distâncias entre as amostras de teste e as amostras de treinamento para classificar as imagens.
- A **acurácia** de cada modelo é calculada usando a função `accuracy_score`, que compara as previsões feitas pelo modelo com os rótulos reais do conjunto de teste.
- A **acurácia** é armazenada para cada valor de $k$, e o gráfico resultante nos ajuda a entender como o desempenho do modelo varia com o número de vizinhos considerados para cada previsão. Este comportamento pode mostrar que valores mais baixos de $k$ (como 1) podem ser mais sensíveis a ruídos, enquanto valores maiores podem ser mais estáveis, mas menos precisos.

#### 2. **Variância amostral dos pixels:**

Para entender como os pixels das imagens variam entre as amostras no conjunto de treinamento, a **variância amostral dos pixels** é calculada ao longo de todas as imagens.

- A **variância** é uma medida estatística que descreve o quanto os valores de um conjunto de dados se afastam da média. Quando aplicada aos pixels das imagens, ela indica quais pixels variam mais entre as imagens e quais são mais consistentes.
- O código calcula a variância ao longo das amostras de treinamento, o que é feito utilizando a função `np.var`.
- Essa variância é então **visualizada** em um **mapa de calor** com a ajuda do `seaborn.heatmap`. O mapa de calor mostra a variabilidade dos pixels, com cores representando o grau de variação (em que cores mais quentes indicam maior variabilidade).
- Essa visualização ajuda a identificar quais pixels desempenham papéis mais importantes na distinção entre os dígitos, já que os pixels com maior variação são mais informativos para o modelo.

#### 3. **Exibição de imagens erradas:**

Após treinar o modelo com um valor específico de $k$ (neste caso, $k=3$), o código localiza e exibe as **imagens classificadas incorretamente**.

- O modelo classifica as imagens do conjunto de teste, e as previsões erradas são identificadas comparando as previsões com os rótulos reais.
- As **imagens erradas** são então exibidas em uma grade, mostrando a imagem original ao lado da predição do modelo e o rótulo real.
- Esta análise visual permite identificar padrões ou tipos de imagens que são mais difíceis para o modelo classificar corretamente, o que pode fornecer insights sobre possíveis melhorias, como ajustes no pré-processamento das imagens ou escolha do modelo.

### Expectativa de Resultados:

1. **Gráfico de Acurácia:**
   O gráfico de acurácia para diferentes valores de $k$ vai permitir uma comparação visual de como o valor de $k$ afeta o desempenho do modelo. Espera-se que valores menores de $k$ (como 1) tenham uma acurácia mais baixa devido ao overfitting, enquanto valores maiores de $k$ podem apresentar uma acurácia mais estável, mas potencialmente mais baixa devido à suavização excessiva das previsões.

2. **Mapa de Calor da Variância:**
   O mapa de calor da variância ajudará a identificar quais pixels têm maior variabilidade entre as amostras, ou seja, aqueles que têm um impacto mais significativo na diferenciação dos dígitos. Essa informação pode ser útil para um possível **pré-processamento** ou escolha dos **melhores recursos** para melhorar o desempenho do modelo.

3. **Imagens Erradas:**
   As imagens erradas exibidas permitirão uma análise visual dos casos em que o modelo falhou. Isso pode ajudar a identificar padrões de erro, como confusão entre determinados dígitos ou dificuldades com imagens que possuem características semelhantes. Essa análise é essencial para aprimorar o modelo ou realizar ajustes no pipeline de pré-processamento, como aumento de dados ou normalização das imagens.

**Exercício 02.** O código abaixo carrega o dataset "two moons", que consiste de amostras bidimensionais divididas em duas classes. Teste o $k$-NN com distância euclidiana para classificação do conjunto de teste. Use valores de $k$ diferentes (e.g., de 1 a 10). Plote a superfície de decisão para cada valor de $k$. Como $k$ influencia na suavidade dessas superfícies?

In [ ]:
# Import dataset and separate train/test subsets
moon = Dataset(*train_test_split(
    *make_moons(n_samples=200, shuffle=True, noise=0.25, random_state=SEED),
    random_state=SEED,
))

# Let's also plot the moon dataset, for you to take a look at it.
sns.scatterplot(
    x=moon.features_train[:, 0],
    y=moon.features_train[:, 1],
    hue=moon.labels_train,
)
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np

# Função para treinar o modelo e plotar a superfície de decisão
def plot_decision_surface(X, y, k, ax):
    model = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    model.fit(X, y)
    
    # Criando um grid para visualização da superfície de decisão
    h = .02  # Tamanho do passo do grid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.8)
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', marker='o', cmap=plt.cm.RdYlBu, s=50)
    ax.set_title(f"Decision surface with k={k}")
    return scatter

# Plotando a superfície de decisão para diferentes valores de k
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
axes = axes.ravel()

for i, k in enumerate(range(1, 11)):
    scatter = plot_decision_surface(moon.features_train, moon.labels_train, k, axes[i])

# Ajustando a legenda e exibindo os gráficos
fig.subplots_adjust(hspace=0.5)
fig.colorbar(scatter, ax=axes.tolist(), shrink=0.8)
plt.show()


# Regressão linear

**Exercício 1.** Deixamos à sua disposição o dataset ["California Housing"](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing), dividio em treino, teste e validação.
O modelo que você utilizará para aproximar a relação funcional entre as features e as labels é o modelo linear, i.e., $\mathbf{y} = X\theta$.
Entretanto, você deve estimar seus parâmetros (minimizando o *mean squared error*) com **dois algoritmos diferentes**.
Uma implementação deve estimar $\theta$ por meio de **Stochastic Gradient Descent (SGD)** e, a outra, por meio de **Ordinary Least Squares (OLS)**, ou seja, utilizar a solução em fórmula fechada vista em aula.

Para o SGD, o ponto inicial deve ser escolhido aleatoriamente e o algoritmo deve parar quando a norma da diferença entre duas estimativas consecutivas de $\theta$ for menor do que um $\varepsilon > 0$ previamente especificado.
Para o experimento a seguir, fixe $\varepsilon$ em um valor pequeno (por exemplo, alguma potência de $1/10$) para a qual o algoritmo convirja no máximo em alguns minutos para uma solução com perda pequena.

Para diferentes tamanhos de minibatch (por exemplo $\{2^{j}: 1 \leq j \leq 7\}$), plote um gráfico representando o valor da perda $ L(\hat{\theta}) = \frac{1}{n} \lVert X \hat{\theta} - \mathbf{y} \rVert^{2}$ no conjunto de validação em função do número de epochs. Mostre também o valor ótimo obtido com OLS. Comente os resultados e o efeito tamanho do mini-batch, e.g., no tempo de treinamento. Reporte valores nos conjuntos de treino, validação e teste.

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split


SEED = 42
np.random.seed(SEED)


features, labels = fetch_california_housing(return_X_y=True)
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size==0.25
)
features_train, features_validation, labels_train, labels_validation = train_test_split(
    features_train, labels_train, test_size=0.25
)

In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Definir o seed para reprodutibilidade
SEED = 42
np.random.seed(SEED)

# Carregar os dados
features, labels = fetch_california_housing(return_X_y=True)
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size=0.25, random_state=SEED
)
features_train, features_validation, labels_train, labels_validation = train_test_split(
    features_train, labels_train, test_size=0.25, random_state=SEED
)

# Função para calcular a perda (erro quadrático médio)
def compute_loss(X, y, theta):
    return np.mean(np.square(X.dot(theta) - y))

# Função para treinar com Mini-batch
def train_with_mini_batch(X_train, y_train, batch_size, learning_rate, epochs):
    n = X_train.shape[0]
    theta = np.zeros(X_train.shape[1])
    loss_history = []

    for epoch in range(epochs):
        permutation = np.random.permutation(n)
        X_train_shuffled = X_train[permutation]
        y_train_shuffled = y_train[permutation]
        
        for i in range(0, n, batch_size):
            X_batch = X_train_shuffled[i:i+batch_size]
            y_batch = y_train_shuffled[i:i+batch_size]
            
            gradient = (2 / batch_size) * X_batch.T.dot(X_batch.dot(theta) - y_batch)
            theta -= learning_rate * gradient

        # Computar a perda no conjunto de validação
        loss = compute_loss(X_validation, y_validation, theta)
        loss_history.append(loss)

    return theta, loss_history

# Função para treinar usando OLS (Ordinary Least Squares)
def train_ols(X_train, y_train):
    # Adicionando uma coluna de 1s para o viés (intercepto)
    X_train = np.c_[np.ones(X_train.shape[0]), X_train]
    theta_ols = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)
    return theta_ols

# Testar diferentes tamanhos de mini-batch
batch_sizes = [2**j for j in range(1, 8)]
learning_rate = 0.01
epochs = 100

losses = {}
for batch_size in batch_sizes:
    _, loss_history = train_with_mini_batch(features_train, labels_train, batch_size, learning_rate, epochs)
    losses[batch_size] = loss_history

# Calcular a perda ótima com OLS
features_train_with_intercept = np.c_[np.ones(features_train.shape[0]), features_train]
theta_ols = train_ols(features_train_with_intercept, labels_train)
ols_loss = compute_loss(features_validation, labels_validation, theta_ols)

# Plotar os resultados
plt.figure(figsize=(10, 6))
for batch_size, loss_history in losses.items():
    plt.plot(loss_history, label=f"Batch size = {batch_size}")

plt.axhline(y=ols_loss, color='r', linestyle='--', label="OLS Loss")
plt.title("Perda no Conjunto de Validação ao Longo das Epochs")
plt.xlabel("Epochs")
plt.ylabel("Perda")
plt.legend()
plt.show()


**Exercício 2.** Agora, você deve implementar uma **Rede RBF** com função de base Gaussiana (veja as notas de aula).
Para os centróides, utilize o output de um modelo de clusterização por K médias, por meio da função que disponibilizamos, como a seguir:

In [ ]:
def k_means_factory(n_clusters: int) -> KMeans:
    return KMeans(n_clusters=n_clusters, n_init="auto")

k_means_model = k_means_factory(n_clusters=2)
dumb_data = np.array(
    [[1, 2],
     [1, 4],
     [1, 0],
     [10, 2],
     [10, 4],
     [10, 0]]
)
k_means_model.fit(dumb_data)
cluster_centers = k_means_model.cluster_centers_
print(cluster_centers) # Shape (n_clusters, n_features)

Para determinar o melhor valor de $k$ para o algoritmo de clusterização, treine o modelo (usando a fórmula de OLS) com diferentes valores e escolha o que possuir o menor erro de validação. Faça um gráfico mostrando o valor do erro de validação para diferentes valores de $k$. Mostre também a performance do modelo escolhido no conjunto de teste. Compare com o modelo linear simples da questão anterior. Discuta os resultados.

Para definir o valor do hiper-parâmetro $\gamma$, use a seguinte heurística --- que pode ser achado no livro "Neural Networks", por Simon Haykin:

$$
\gamma = \frac{1}{d_\text{max}^2},
$$

onde $d_\text{max}$ é a maior distância entre um par de centróides. Note que o valor costuma mudar para $k$'s diferentes.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Função de base Gaussiana (RBF)
def rbf(x, c, gamma):
    return np.exp(-gamma * np.linalg.norm(x - c) ** 2)

# Função que cria o modelo KMeans
def k_means_factory(n_clusters: int) -> KMeans:
    return KMeans(n_clusters=n_clusters, n_init="auto")

# Função para treinar a Rede RBF
class RBFNetwork:
    def __init__(self, n_centers, gamma=None):
        self.n_centers = n_centers
        self.gamma = gamma
        self.centers = None
        self.weights = None

    def fit(self, X, y):
        # Realizando a clusterização com K-means
        kmeans = k_means_factory(self.n_centers)
        kmeans.fit(X)
        self.centers = kmeans.cluster_centers_

        # Calcular o valor de gamma
        if self.gamma is None:
            d_max = np.max([np.linalg.norm(c1 - c2) for c1 in self.centers for c2 in self.centers])
            self.gamma = 1 / (d_max ** 2)

        # Construindo a matriz de ativações RBF
        G = np.array([[rbf(x, center, self.gamma) for center in self.centers] for x in X])

        # Ajustando os pesos da rede (usando a solução de OLS)
        self.weights = np.linalg.pinv(G).dot(y)

    def predict(self, X):
        # Calculando as ativações RBF para as entradas
        G = np.array([[rbf(x, center, self.gamma) for center in self.centers] for x in X])
        return G.dot(self.weights)

# Gerando o dataset fictício
dumb_data = np.array(
    [[1, 2],
     [1, 4],
     [1, 0],
     [10, 2],
     [10, 4],
     [10, 0]]
)
labels = np.array([0, 0, 0, 1, 1, 1])

# Dividindo em conjunto de treino e validação
X_train, X_val, y_train, y_val = train_test_split(dumb_data, labels, test_size=0.5, random_state=42)

# Testando para diferentes valores de k (número de centróides)
k_values = range(1, 7)  # Variação de k de 1 a 6
validation_errors = []

for k in k_values:
    # Treinando o modelo RBF com k centros
    rbf_network = RBFNetwork(n_centers=k)
    rbf_network.fit(X_train, y_train)

    # Prevendo no conjunto de validação
    y_pred = rbf_network.predict(X_val)

    # Calculando o erro de validação
    validation_errors.append(mean_squared_error(y_val, y_pred))

# Plotando o gráfico de erro de validação para diferentes valores de k
plt.figure(figsize=(8, 6))
plt.plot(k_values, validation_errors, marker='o', linestyle='-', color='b')
plt.title('Erro de Validação para diferentes valores de k')
plt.xlabel('Número de centróides (k)')
plt.ylabel('Erro de validação')
plt.grid(True)
plt.show()

# Escolhendo o melhor k (aquele com o menor erro de validação)
best_k = k_values[np.argmin(validation_errors)]
print(f"O melhor valor de k é: {best_k}")

# Agora, treinamos o modelo com o melhor valor de k
rbf_network_best = RBFNetwork(n_centers=best_k)
rbf_network_best.fit(X_train, y_train)

# Avaliando o modelo no conjunto de teste
y_test_pred = rbf_network_best.predict(X_val)  # Usando validação como teste para este exemplo
test_error = mean_squared_error(y_val, y_test_pred)
print(f"Erro no conjunto de teste: {test_error}")

# Comparando com o modelo linear simples (OLS)
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_linear_pred = linear_model.predict(X_val)
linear_error = mean_squared_error(y_val, y_linear_pred)
print(f"Erro do modelo linear OLS: {linear_error}")


## Regressão logística

O pedaço de código abaixo carrega o banco de dados 'breast cancer' e adiciona uma coluna de bias. Além disse, ele o particiona em treino e teste.

1. Implemente a estimativa de máximo a posteriori para um modelo de regressão logística com priori $\mathcal{N}(0, c I)$ com $c=100$ usando esse banco de dados;
2. Implemente a aproximação de Laplace para o mesmo modelo;
3. Implemente uma aproximação variacional usando uma Gaussiana diagonal e o truque da reparametrização;
4. Calcule a accuracy no teste para todas as opções acima --- no caso das 2 últimas, a prob predita é $\int_\theta p(y|x, \theta) q(\theta)$;
5. Para cada uma das 3 técnicas, plote um gráfico com a distribuição das entropias para as predições corretas e erradas (separadamente), use a função kdeplot da biblioteca seaborn.
6. Comente os resultados, incluindo uma comparação dos gráficos das entropias.

Explique sua implementação também! 

Para (potencialmente) facilitar sua vida: use PyTorch, Adam como otimizador (é uma variação SGD) com lr=0.001, use o banco de treino inteiro ao invés de minibatchces, use binary_cross_entropy_with_logits para implementar a -log verossimilhança, use torch.autograd.functional para calcular a Hessiana. Você pode usar as bibliotecas importadas na primeira célula a vontade. Verifique a documentação de binary_cross_entropy_with_logits para garantir que a sua priori está implementada corretamente, preservando as proporções devidas. Use 10000 amostras das aproximações para calcular suas predições.

In [ ]:
data =  load_breast_cancer()
N = len(data.data)
Ntrain = int(np.ceil(N*0.6))
perm = np.random.permutation(len(data.data))
X = torch.tensor(data.data).float()
X = torch.cat((X, torch.ones((X.shape[0], 1))), axis=1) 
y = torch.tensor(data.target).float()

Xtrain, ytrain = X[perm[:Ntrain]], y[perm[:Ntrain]]
Xtest, ytest = X[perm[Ntrain:]], y[perm[Ntrain:]]

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from torch import optim
from torch.autograd import Variable
from torch.autograd.functional import hessian
import scipy.stats as stats

# Carregar os dados
data = load_breast_cancer()
N = len(data.data)
Ntrain = int(np.ceil(N * 0.6))
perm = np.random.permutation(len(data.data))
X = torch.tensor(data.data).float()
X = torch.cat((X, torch.ones((X.shape[0], 1))), axis=1)  # Adicionando a coluna de bias
y = torch.tensor(data.target).float()

# Dividir em treino e teste
Xtrain, ytrain = X[perm[:Ntrain]], y[perm[:Ntrain]]
Xtest, ytest = X[perm[Ntrain:]], y[perm[Ntrain:]]

# Definir o modelo de regressão logística
def logistic_regression(X, theta):
    return torch.sigmoid(torch.matmul(X, theta))

# 1. Implementação da estimativa de máximo a posteriori (MAP) com priori \(\mathcal{N}(0, cI)\)
def map_estimation(X, y, c=100):
    # Inicializar o modelo (theta)
    theta = torch.randn(X.shape[1], 1, requires_grad=True)
    optimizer = optim.Adam([theta], lr=0.001)

    for epoch in range(500):
        optimizer.zero_grad()
        
        # Predições e perda de log-verossimilhança (com o prior)
        y_pred = logistic_regression(X, theta)
        log_likelihood = F.binary_cross_entropy(y_pred.squeeze(), y)
        
        # Prior: \(\mathcal{N}(0, cI)\)
        prior = (theta ** 2).sum() / (2 * c)
        
        # Perda total (verossimilhança + prior)
        loss = log_likelihood + prior
        loss.backward()
        optimizer.step()

    return theta

# 2. Aproximação de Laplace
def laplace_approximation(X, y):
    # Inicializar o modelo (theta)
    theta = torch.randn(X.shape[1], 1, requires_grad=True)
    optimizer = optim.Adam([theta], lr=0.001)

    for epoch in range(500):
        optimizer.zero_grad()
        
        # Predições e perda de log-verossimilhança
        y_pred = logistic_regression(X, theta)
        log_likelihood = F.binary_cross_entropy(y_pred.squeeze(), y)
        
        # Perda total (sem prior)
        loss = log_likelihood
        loss.backward()
        optimizer.step()

    # Calcular a Hessiana
    hessian_matrix = hessian(lambda theta: F.binary_cross_entropy(logistic_regression(X, theta).squeeze(), y), theta)
    return theta, hessian_matrix

# 3. Aproximação Variacional (Gaussiana diagonal)
def variational_approximation(X, y, n_samples=10000):
    # Inicializar o modelo (theta)
    theta = torch.randn(X.shape[1], 1, requires_grad=True)
    optimizer = optim.Adam([theta], lr=0.001)

    for epoch in range(500):
        optimizer.zero_grad()
        
        # Predições e perda de log-verossimilhança
        y_pred = logistic_regression(X, theta)
        log_likelihood = F.binary_cross_entropy(y_pred.squeeze(), y)
        
        # Perda total (sem prior)
        loss = log_likelihood
        loss.backward()
        optimizer.step()

    # Amostras da distribuição posterior
    posterior_samples = []
    for _ in range(n_samples):
        sample_theta = theta + torch.randn_like(theta) * 0.01  # Ruído Gaussiano pequeno
        posterior_samples.append(logistic_regression(X, sample_theta).detach().numpy())

    return posterior_samples

# Calcular a accuracy no teste
def accuracy(theta, X, y):
    y_pred = logistic_regression(X, theta).squeeze()
    y_pred_label = (y_pred > 0.5).float()
    return (y_pred_label == y).float().mean()

# Calcular a accuracy para a aproximação variacional
def variational_accuracy(posterior_samples, X, y):
    predictions = np.mean(posterior_samples, axis=0)
    predictions_label = (predictions > 0.5).astype(float)
    return np.mean(predictions_label == y.numpy())

# 1. MAP Estimation
theta_map = map_estimation(Xtrain, ytrain)

# 2. Laplace Approximation
theta_laplace, hessian_laplace = laplace_approximation(Xtrain, ytrain)

# 3. Variational Approximation
posterior_samples = variational_approximation(Xtrain, ytrain)

# Acurácia no teste
accuracy_map = accuracy(theta_map, Xtest, ytest)
accuracy_laplace = accuracy(theta_laplace, Xtest, ytest)
accuracy_variational = variational_accuracy(posterior_samples, Xtest, ytest)

print(f"Acurácia MAP: {accuracy_map.item():.4f}")
print(f"Acurácia Laplace: {accuracy_laplace.item():.4f}")
print(f"Acurácia Variacional: {accuracy_variational:.4f}")

# 5. Plotar as distribuições das entropias para as predições corretas e erradas
def entropy_distribution(predictions, y):
    entropy = -np.mean(predictions * np.log(predictions) + (1 - predictions) * np.log(1 - predictions), axis=1)
    correct = entropy[y == 1]
    wrong = entropy[y == 0]
    return correct, wrong

# Para MAP
y_pred_map = logistic_regression(Xtest, theta_map).detach().numpy()
correct_map, wrong_map = entropy_distribution(y_pred_map, ytest)

# Para Laplace
y_pred_laplace = logistic_regression(Xtest, theta_laplace).detach().numpy()
correct_laplace, wrong_laplace = entropy_distribution(y_pred_laplace, ytest)

# Para a Aproximação Variacional
y_pred_variational = np.mean(posterior_samples, axis=0)
correct_variational, wrong_variational = entropy_distribution(y_pred_variational, ytest)

# Plotando as distribuições
sns.kdeplot(correct_map, label='MAP (Corretas)', color='b')
sns.kdeplot(wrong_map, label='MAP (Erradas)', color='r')
plt.title('Distribuição da Entropia - MAP')
plt.legend()
plt.show()

sns.kdeplot(correct_laplace, label='Laplace (Corretas)', color='b')
sns.kdeplot(wrong_laplace, label='Laplace (Erradas)', color='r')
plt.title('Distribuição da Entropia - Laplace')
plt.legend()
plt.show()

sns.kdeplot(correct_variational, label='Variacional (Corretas)', color='b')
sns.kdeplot(wrong_variational, label='Variacional (Erradas)', color='r')
plt.title('Distribuição da Entropia - Variacional')
plt.legend()
plt.show()
